In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
import scipy.optimize as opt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import multiprocessing
from pathlib import Path
import pathlib
from tqdm.notebook import tqdm
INPUT = '../input/google-smartphone-decimeter-challenge/'
root = Path(INPUT)

In [2]:
def ecef2lla(x, y, z):
    # x, y and z are scalars or vectors in meters
    x = np.array([x]).reshape(np.array([x]).shape[-1], 1)
    y = np.array([y]).reshape(np.array([y]).shape[-1], 1)
    z = np.array([z]).reshape(np.array([z]).shape[-1], 1)

    a=6378137
    a_sq=a**2
    e = 8.181919084261345e-2
    e_sq = 6.69437999014e-3

    f = 1/298.257223563
    b = a*(1-f)

    # calculations:
    r = np.sqrt(x**2 + y**2)
    ep_sq  = (a**2-b**2)/b**2
    ee = (a**2-b**2)
    f = (54*b**2)*(z**2)
    g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
    c = (e_sq**2)*f*r**2/(g**3)
    s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
    p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
    q = np.sqrt(1 + 2*p*e_sq**2)
    r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
    u = np.sqrt((r - e_sq*r_0)**2 + z**2)
    v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
    z_0 = (b**2)*z/(a*v)
    h = u*(1 - b**2/(a*v))
    phi = np.arctan((z + ep_sq*z_0)/r)
    lambd = np.arctan2(y, x)

    return phi*180/np.pi, lambd*180/np.pi, h

def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
      np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [3]:
def make_gt(recal):
    if recal:
        p = pathlib.Path(INPUT)
        gt_files = list(p.glob('train/*/*/ground_truth.csv'))
        print('ground_truth.csv count : ', len(gt_files))

        gts = []
        for gt_file in tqdm(gt_files):
            gts.append(pd.read_csv(gt_file))
        ground_truth = pd.concat(gts)
        ground_truth.to_csv(root / 'gt.csv',index=False)
    else:
        ground_truth = pd.read_csv(root / 'gt.csv')
    return ground_truth
    
gt = make_gt(recal=True)

ground_truth.csv count :  73


  0%|          | 0/73 [00:00<?, ?it/s]

In [4]:
def percentile50(x):
    return np.percentile(x, 50)
def percentile95(x):
    return np.percentile(x, 95)

def get_train_score(df, gt):
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    # calc_distance_error
    df['err'] = calc_haversine(df['latDeg_gt'], df['lngDeg_gt'], df['latDeg'], df['lngDeg'])
    # calc_evaluate_score
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    res = df.groupby('phone')['err'].agg([percentile50, percentile95])
    res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2 
    score = res['p50_p90_mean'].mean()
    return score

def get_train_score_df(df, gt):
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='left')
    # calc_distance_error
    df['err'] = calc_haversine(df['latDeg_gt'], df['lngDeg_gt'], df['latDeg'], df['lngDeg'])
    return df

In [5]:
def gnss_log_to_dataframes(path):
    '''Load GNSS Log'''
    print('Loading ' + path, flush = True)
    gnss_section_names = {'Raw', 'UncalAccel', 'UncalGyro', 'UncalMag', 'Fix', 'Status', 'OrientationDeg'}
    with open(path) as f_open:
        datalines = f_open.readlines()

    datas = {k: [] for k in gnss_section_names}
    gnss_map = {k: [] for k in gnss_section_names}
    for dataline in datalines:
        is_header = dataline.startswith('#')
        dataline = dataline.strip('#').strip().split(',')
        # skip over notes, version numbers, etc
        if is_header and dataline[0] in gnss_section_names:
            gnss_map[dataline[0]] = dataline[1:]
        elif not is_header:
            datas[dataline[0]].append(dataline[1:])

    results = dict()
    for k, v in datas.items():
        results[k] = pd.DataFrame(v, columns=gnss_map[k])
    # pandas doesn't properly infer types from these lists by default
    for k, df in results.items():
        for col in df.columns:
            if col == 'CodeType':
                continue
            results[k][col] = pd.to_numeric(results[k][col])

    return results

In [6]:
# apply tips1
# _derivedのmillisSinceGpsEpochが次のepoch(rawの)を示しているので元に戻す
def apply_tips1(raw_df, derived_df):
    # Create a new column in df_raw that corresponds to derivedの['millisSinceGpsEpoch']
    raw_df['millisSinceGpsEpoch'] = np.floor((raw_df['TimeNanos'] - raw_df['FullBiasNanos']) / 1000000.0).astype(int)
        
    # Change each value in df_derived['MillisSinceGpsEpoch'] to be the prior epoch.
    raw_timestamps = raw_df['millisSinceGpsEpoch'].unique()
    derived_timestamps = derived_df['millisSinceGpsEpoch'].unique()

    # The timestamps in derived are one epoch ahead. We need to map each epoch
    # in derived to the prior one (in Raw).
    indexes = np.searchsorted(raw_timestamps, derived_timestamps)
    from_t_to_fix_derived = dict(zip(derived_timestamps, raw_timestamps[indexes-1]))
    derived_df['millisSinceGpsEpoch'] = np.array(list(map(lambda v: from_t_to_fix_derived[v], derived_df['millisSinceGpsEpoch'])))
    return derived_df

# apply tips5
# derivedの重複している行を削除
def apply_tips5(derived_df):
    delta_millis = derived_df['millisSinceGpsEpoch'] - derived_df['receivedSvTimeInGpsNanos'] / 1e6
    where_good_signals = (delta_millis > 0) & (delta_millis < 300)
    return derived_df[where_good_signals]

In [7]:
output_dir = '../input/derived/'
os.makedirs(output_dir, exist_ok=True)

In [8]:
def estimate_train_position_by_derived(args):
    (collection_name, phone_name), base_df = args
    # Train df here only contains one collection and one measurement
    derived_df = pd.read_csv(root / f"train/{collection_name}/{phone_name}/{phone_name}_derived.csv")
    gnss_df = gnss_log_to_dataframes(str(root / f"train/{collection_name}/{phone_name}/{phone_name}_GnssLog.txt"))
    raw_df = gnss_df['Raw']

    # epochを合わせる
    derived_df = apply_tips1(raw_df, derived_df)
    derived_df = apply_tips5(derived_df)
    derived_df = derived_df.sort_values('millisSinceGpsEpoch')
    
    # pseudorangeの修正
    derived_df['correctedPrM'] = derived_df.apply(lambda r: r.rawPrM + r.satClkBiasM - r.isrbM - r.ionoDelayM - r.tropoDelayM,axis=1)
    
    # 伝播時間=擬似距離/光速
    # 受信時刻と送信時刻の差分となる
    light_speed = 299_792_458
    derived_df['transmissionTimeSeconds'] = derived_df['correctedPrM'] / light_speed

    # Compute true sat positions at arrival time
    # 到着までに衛星位置が移動しているのでこれを補正
    omega_e = 7.2921151467e-5
    derived_df['xSatPosMRotated'] = \
        np.cos(omega_e * derived_df['transmissionTimeSeconds']) * derived_df['xSatPosM'] \
        + np.sin(omega_e * derived_df['transmissionTimeSeconds']) * derived_df['ySatPosM']

    derived_df['ySatPosMRotated'] = \
        - np.sin(omega_e * derived_df['transmissionTimeSeconds']) * derived_df['xSatPosM'] \
        + np.cos(omega_e * derived_df['transmissionTimeSeconds']) * derived_df['ySatPosM']

    derived_df['zSatPosMRotated'] = derived_df['zSatPosM']

    derived_df['uncertaintyWeight'] = 1 / derived_df['rawPrUncM']

    output_df = pd.DataFrame()
    d_list = []
    x_list = []
    y_list = []
    z_list = []
    epoch_list = []
    for epoch, df in derived_df.groupby('millisSinceGpsEpoch'): 
        # Corrected pseudorange according to data instructions
        # Time it took for signal to travel
        # Start point for the optimiser

        # 最小2乗法による座標の推定
        x0 = [1,1,1,1]
        opt_res = opt.least_squares(distance, x0, 
        args=(
            df['xSatPosMRotated'].values, 
            df['ySatPosMRotated'].values, 
            df['zSatPosMRotated'].values,
            df['correctedPrM'].values, 
            df['uncertaintyWeight'].values))

        # opt_res = opt.basinhopping(distance, x0, stepsize=2,
        # minimizer_kwargs={"args":(
        #     df['xSatPosMRotated'].values, 
        #     df['ySatPosMRotated'].values, 
        #     df['zSatPosMRotated'].values,
        #     df['correctedPrM'].values, 
        #     df['uncertaintyWeight'].values)}
        #     )
        # Optimiser yields a position in the ECEF coordinates
        opt_res_pos = opt_res.x
        d = distance(opt_res_pos, df['xSatPosMRotated'], df['ySatPosMRotated'], df['zSatPosMRotated'], df['correctedPrM'], df['uncertaintyWeight'])

        # ECEF position to lat/long
        wls_estimated_pos = ecef2lla(*opt_res_pos[:3])
        wls_estimated_pos = np.squeeze(wls_estimated_pos)
        d_list.append(d)
        x_list.append(wls_estimated_pos[0])
        y_list.append(wls_estimated_pos[1])
        z_list.append(wls_estimated_pos[2])
        epoch_list.append(epoch)

    output_df["latDeg"] = x_list
    output_df["lngDeg"] = y_list
    output_df['heightAboveWgs84EllipsoidM'] = z_list
    output_df["dist"] = d_list
    output_df['millisSinceGpsEpoch'] = epoch_list
    output_df['collectionName'] = collection_name
    output_df['phoneName'] = phone_name

    output_df.to_csv(output_dir + f'{collection_name}_{phone_name}_derived.csv', index=False)
    return output_df

In [9]:
# Set up least squares methods
def distance(x, satx, saty, satz, prm, weight):
    
    satx = satx - x[0]
    saty = saty - x[1]
    satz = satz - x[2]

    d = weight * (np.sqrt((satx**2 + saty**2 +satz**2)) + x[3] - prm)
    return d

# def distance_fixed_satpos(x, row):

#     return distance(df[['xSatPosMRotated', 'ySatPosMRotated', 'zSatPosMRotated', 'correctedPrM', 'uncertaintyWeight']], x)

In [10]:
base_train = pd.read_csv(root/ 'baseline_locations_train.csv')
base_train.loc[:,['px','py','pz']] = 0
base_train.head()

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone,px,py,pz
0,2020-05-14-US-MTV-1,Pixel4,1273529463442,37.423575,-122.094091,-34.06,2020-05-14-US-MTV-1_Pixel4,0,0,0
1,2020-05-14-US-MTV-1,Pixel4,1273529464442,37.423578,-122.094101,-33.29,2020-05-14-US-MTV-1_Pixel4,0,0,0
2,2020-05-14-US-MTV-1,Pixel4,1273529465442,37.423573,-122.094111,-30.99,2020-05-14-US-MTV-1_Pixel4,0,0,0
3,2020-05-14-US-MTV-1,Pixel4,1273529466442,37.423583,-122.094121,-32.83,2020-05-14-US-MTV-1_Pixel4,0,0,0
4,2020-05-14-US-MTV-1,Pixel4,1273529467442,37.423579,-122.094114,-34.49,2020-05-14-US-MTV-1_Pixel4,0,0,0


In [11]:
import multiprocessing

gr = base_train.groupby(['collectionName','phoneName'])
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    dfs = pool.imap_unordered(estimate_train_position_by_derived, gr)
    dfs = tqdm(dfs, total=len(gr))
    dfs = list(dfs)
all_derived_df = pd.concat(dfs).sort_values(['collectionName', 'phoneName', 'millisSinceGpsEpoch']).reset_index(drop=True)     

  0%|          | 0/73 [00:00<?, ?it/s]

Loading ../input/google-smartphone-decimeter-challenge/train/2020-05-14-US-MTV-2/Pixel4XLModded/Pixel4XLModded_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2020-06-05-US-MTV-1/Pixel4XLModded/Pixel4XLModded_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2020-05-29-US-MTV-1/Pixel4/Pixel4_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2020-05-21-US-MTV-1/Pixel4/Pixel4_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2020-05-29-US-MTV-2/Pixel4XL/Pixel4XL_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2020-05-14-US-MTV-1/Pixel4XLModded/Pixel4XLModded_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2020-05-29-US-MTV-1/Pixel4XL/Pixel4XL_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2020-06-04-US-MTV-1/Pixel4/Pixel4_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2020-05-29-US-MTV-2/Pixel4/

KeyboardInterrupt: 

In [ ]:
all_derived_df["svid"].unique()

In [22]:
from tqdm.notebook import tqdm
df_list = []
count = 0
for (collection_name, phone_name), base_df in tqdm(base_train.groupby(['collectionName','phoneName'])):
    # break
    print("")
    print(collection_name, phone_name)
    base_df = base_df.sort_values('millisSinceGpsEpoch')
    target_gt = gt[(gt['collectionName']==collection_name)&(gt['phoneName']==phone_name)].sort_values('millisSinceGpsEpoch').reset_index(drop=True)
    # 上で作成したderivedデータの読み込み
    derived_df = pd.read_csv(output_dir + f'{collection_name}_{phone_name}_derived.csv')
    derived_df = derived_df[~derived_df["millisSinceGpsEpoch"].duplicated()].sort_values("millisSinceGpsEpoch").reset_index(drop=True)
    derived_df = derived_df.rename(columns={"latDeg":"_latDeg", "lngDeg":"_lngDeg"})
    print(base_df.shape, target_gt.shape, derived_df.shape)
    base_score = get_train_score(base_df, target_gt)
    print("baseline:", base_score)

    derived_df = pd.merge_asof(base_df, derived_df[["millisSinceGpsEpoch", "_latDeg", "_lngDeg"]], on=["millisSinceGpsEpoch"], tolerance=10, direction='nearest')

    # データが欠損しているところはbaselineで置き換える
    derived_df.loc[derived_df["_latDeg"].isna(), "_latDeg"] = derived_df.loc[derived_df["_latDeg"].isna(), "latDeg"].values
    derived_df.loc[derived_df["_lngDeg"].isna(), "_lngDeg"] = derived_df.loc[derived_df["_lngDeg"].isna(), "lngDeg"].values

    derived_df = derived_df.drop(["latDeg", "lngDeg"], axis=1).rename(columns={"_latDeg":"latDeg","_lngDeg":"lngDeg"})
    df_list.append(derived_df)
    derived_score = get_train_score(derived_df, target_gt)
    print("derived:", derived_score)
    # if count == 1:
    #     break
    # count += 1
corrected_base_df = pd.concat(df_list).reset_index(drop=True)

  0%|          | 0/73 [00:00<?, ?it/s]


2020-05-14-US-MTV-1 Pixel4
(1740, 10) (1740, 11) (1738, 7)
baseline: 2.100601745976844
derived: 2.171155340633557

2020-05-14-US-MTV-1 Pixel4XLModded
(1746, 10) (1746, 11) (1744, 7)
baseline: 3.169948400933192
derived: 3.673965540406078

2020-05-14-US-MTV-2 Pixel4
(1770, 10) (1770, 11) (1768, 7)
baseline: 2.0838364333971855
derived: 2.160422615755044

2020-05-14-US-MTV-2 Pixel4XLModded
(577, 10) (577, 11) (570, 7)
baseline: 7.362608642825408
derived: 8.275379202614172

2020-05-21-US-MTV-1 Pixel4
(2031, 10) (2031, 11) (1958, 7)
baseline: 3.2407210547287955
derived: 3.467088590798185

2020-05-21-US-MTV-2 Pixel4
(1965, 10) (1965, 11) (1963, 7)
baseline: 2.1194285544204288
derived: 2.411436724863336

2020-05-21-US-MTV-2 Pixel4XL
(1794, 10) (1794, 11) (1742, 7)
baseline: 2.27732233777214
derived: 2.492302650630357

2020-05-29-US-MTV-1 Pixel4
(1913, 10) (1913, 11) (1911, 7)
baseline: 3.669069401945867
derived: 3.7622486890577154

2020-05-29-US-MTV-1 Pixel4XL
(1912, 10) (1912, 11) (1910, 7)


In [53]:
# collection_name = '2021-01-05-US-SVL-1' 
# phone_name = 'Mi8'
# derived_df = pd.read_csv(output_dir + f'{collection_name}_{phone_name}_derived.csv')
# base_df = base_train[(base_train['collectionName']==collection_name)&(base_train['phoneName']==phone_name)]

In [ ]:
# import seaborn as sns
# sns.scatterplot(y='px', x='millisSinceGpsEpoch', data=derived_df[derived_df['signalType']=='GPS_L1'])

In [ ]:
# import seaborn as sns
# sns.scatterplot(y='latDeg', x='millisSinceGpsEpoch', data=base_df)

In [ ]:
# derived_df['px'].plot('signalType')

In [23]:
get_train_score(base_train, gt)

5.287970649047861

In [24]:
get_train_score(corrected_base_df, gt)

5.921256745231846

In [58]:
corrected_base_df.shape, base_train.shape

((131342, 10), (131342, 10))

In [60]:
mean_df = base_train.copy()
mean_df['latDeg'] = corrected_base_df['latDeg']*0.2 + base_train['latDeg']*0.8
mean_df['lngDeg'] = corrected_base_df['lngDeg']*0.2 + base_train['lngDeg']*0.8
get_train_score(mean_df, gt)

5.343350555666337